# <span style='font-family:"Times New Roman"'> <span styel=''> **OncoKB Annotation**

*Emile Cohen* 
    
*March 2020*

**Goal:** In this notebook, we call the OncoKB API to get the annotation at variant level for MSK-Impact cohort. 

---

In [1]:
%run -i '../../utils/setup_environment.ipy'

import warnings
import requests
import click
import os
import json
import time
from tqdm import tqdm_notebook as tqdm
warnings.filterwarnings('ignore')

data_path = '../../data/'

Setup environment... done!


<span style="color:green">✅ Working on **mskimpact_env** conda environment.</span>

# API Call

---
To call the API, we need a token to access the API, that could be found after logging in in OncoKB (https://www.oncokb.org/dataAccess) in *Account Settings*.

My Token is : **f08eeec6-fe2e-4414-9352-df71e2c1db32**

---

## Useful functions to call OncoKB API

In [109]:
def retry_request(method, **kwargs):
    """Retry request operation multiple times."""
    url = kwargs.get('url') or ""
    click.secho(f"Querying to {url}", fg="green")
    for i in [0.2, 1, 5]:  # attempt some retries
        try:
            error = None
            response = getattr(requests, method)(verify=False, **kwargs)
        except requests.exceptions.RequestException as request_error:  # pragma: no cover
            error = request_error
            response = None
        if response is not None and not str(response.status_code).startswith("50"):
            break
        else:  # pragma: no cover
            msg = f"Request to {url} failed with error: {error}, retrying in {i}s..."
            click.secho(msg, fg="yellow", err=True)
            time.sleep(i)
    return response


ONCOKB_ENDPOINTS = {
    "api_info": "https://www.oncokb.org/api/v1/info",
    "cancer_genes": "https://www.oncokb.org/api/v1/utils/cancerGeneList",
    "annotate": "https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange",
}


def oncokb_query(endpoint, method="get", data=None):
    """Make authenticated calls to newest oncokb api to gen cancer genes."""
    oncokb_token = "f08eeec6-fe2e-4414-9352-df71e2c1db32"
    if not oncokb_token:
        raise click.UsageError(
            "oncoKb authentication token is missing. "
            "Add it as enviromental variable:"
            "\n\texport ONCOKB_TOKEN=<your-oncokb-token>"
            "\nOr add them to a .env file."
        )
    headers = {
        "Authorization": f"Bearer {oncokb_token}",
        "Content-Type": "application/json",
    }
    response = retry_request(method, url=endpoint, headers=headers, data=data)
    if not response.ok:
        raise click.UsageError(f"OncoKb query failed. Reason: {response.reason}")
    return json.loads(response.content)


def get_oncokb_treatments(df, genomic_cols, tumor_type):
    """Return df with extra columns for Drug and Evidence Level."""
    chrom_col, start_col, end_col, ref_col, alt_col = genomic_cols
    genomic_changes = [
        [v[chrom_col], v[start_col], v[end_col], v[ref_col], v[alt_col]]
        for v in df.T.to_dict().values()
    ]
    data = [
        {
            "genomicLocation": ",".join(map(str, i)),
            "tumorType": tumor_type,
            "id": "",
            "evidenceTypes": [],
        }
        for i in genomic_changes
    ]
    annotated_variants = oncokb_query(
        ONCOKB_ENDPOINTS["annotate"], method="post", data=json.dumps(data)
    )
    return annotated_variants


def annotate_variants(df): 
    data = [
    {
      "genomicLocation": "{},{},{},{},{}".format(
        v["Chromosome"], int(v["Start_Position"]), int(v["End_Position"]), v["Reference_Allele"], v["Variant_Allele"]
      ),
      "tumorType": v["Cancer_Type"],
      "id": "",
      "evidenceTypes": [],
    }
    for v in df.T.to_dict().values()
    ]
    try: annotated_variants = oncokb_query(ONCOKB_ENDPOINTS["annotate"], method="post", data=json.dumps(data))
    except: return [['None', 'None', 'None', 'None']]
    
    return [[v['mutationEffect']['knownEffect'], v['oncogenic'],v['vus'], v['hotspot']] for v in annotated_variants]

def get_oncokb_genes():
    """Get list of genes and biomarker incindence levels from oncokb."""
    return oncokb_query(ONCOKB_ENDPOINTS["cancer_genes"])


### Use Chunks and Multiprocessing to download faster

In [105]:
maf_cohort = pd.read_pickle(data_path + 'merged_data/maf_cohort.pkl')
maf_cohort = maf_cohort.dropna(subset=["Chromosome", "Start_Position", "End_Position", "Reference_Allele", "Variant_Allele"])

In [41]:
%%time
query = annotate_variants(maf_cohort)

Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
CPU times: user 23.1 s, sys: 553 ms, total: 23.7 s
Wall time: 24 s


In [66]:
new_data = pd.DataFrame(query, columns=['mutationEffect', 'oncogenic', 'vus', 'hotspot'])#['mutationEffect']['knownEffect']

new_data

,mutationEffect,oncogenic,vus,hotspot
0,Gain-of-function,Oncogenic,False,True
1,Gain-of-function,Oncogenic,False,True


In [111]:
import time

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


def fill_with_queries(df: pd.DataFrame, chunk_size: int):
    
    maf_chunks = list(chunks(df, chunk_size))[92300:]
    list_ = [chunk_size*x for x in list(range(len(maf_chunks)))] 

    for i, chunk in tqdm(zip(list_, maf_chunks)):
        #print(i, len(chunk))
        df_chunk = df[92300+i:92300+i+chunk_size]
        query = annotate_variants(chunk)
        new_data = pd.DataFrame(query, columns=['mutationEffect', 'oncogenic', 'vus', 'hotspot'])

        for column in ['mutationEffect', 'oncogenic', 'vus', 'hotspot']:
            df_chunk[column] = new_data[column]
        
        df_chunk.to_pickle(data_path + 'annotation/maf_cohort_annotated_{}_{}.pkl'.format(92300+i,92300+i+chunk_size))
        print(92300+i)
        #time.sleep(60*10)

    return df

In [ ]:
%%time
maf_cohort= pd.read_pickle(data_path + 'merged_data/maf_cohort.pkl')

# Deleting NaN lines
maf_cohort = maf_cohort.dropna(subset=["Chromosome", "Start_Position", "End_Position", "Reference_Allele", "Variant_Allele"])
maf_cohort = maf_cohort[maf_cohort['Chromosome'] != 'NA'][maf_cohort['Start_Position'] != 'NA'][maf_cohort['End_Position'] != 'NA'][maf_cohort['Reference_Allele'] != 'NA'][maf_cohort['Variant_Allele'] != 'NA']
maf_cohort['mutationEffect'] = 'NaN' ; maf_cohort['oncogenic'] = 'NaN' ; maf_cohort['vus'] = 'NaN' ; maf_cohort['hotspot'] = 'NaN'
#maf_cohort[["Chromosome", "Start_Position", "End_Position", "Reference_Allele", "Variant_Allele"]][92301:92302]
fill_with_queries(maf_cohort, 1)

Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92300
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange


Request to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange failed with error: None, retrying in 0.2s...
Request to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange failed with error: None, retrying in 1s...
Request to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange failed with error: None, retrying in 5s...


92301
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92302
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92303
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92304
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92305
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92306
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92307
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92308
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92309
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92310
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92311
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92312
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
9231

92391
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92392
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92393
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92394
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92395
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92396
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92397
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92398
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92399
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92400
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92401
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92402
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
9240

92481
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92482
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92483
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92484
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92485
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92486
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92487
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92488
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92489
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92490
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92491
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
92492
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
9249

...

94911
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94912
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94913
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94914
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94915
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94916
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94917
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94918
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94919
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94920
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94921
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
94922
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
9492

95001
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95002
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95003
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95004
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95005
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95006
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95007
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95008
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95009
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95010
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95011
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95012
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
9501

95091
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95092
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95093
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95094
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95095
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95096
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95097
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95098
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95099
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95100
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95101
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95102
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
9510

95181
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95182
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95183
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95184
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95185
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95186
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95187
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95188
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95189
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95190
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95191
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95192
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
9519

95271
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95272
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95273
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95274
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95275
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95276
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95277
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95278
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95279
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95280
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95281
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
95282
Querying to https://www.oncokb.org/api/v1/annotate/mutations/byGenomicChange
9528

In [78]:
test = maf_cohort[97500:97600][["Chromosome", "Start_Position", "End_Position", "Reference_Allele", "Variant_Allele"]]
maf_cohort[maf_cohort['Chromosome'] != 'NA'][maf_cohort['Start_Position'] != 'NA'][maf_cohort['End_Position'] != 'NA'][maf_cohort['Reference_Allele'] != 'NA'][maf_cohort['Variant_Allele'] != 'NA']

,Chromosome,Start_Position,End_Position,Reference_Allele,Variant_Allele
97946,2,25982440.0,25982440.0,G,A
97947,9,98270529.0,98270530.0,-,C
97948,6,106553389.0,106553389.0,G,A
97949,4,66356133.0,66356133.0,C,T
97950,7,152345760.0,152345760.0,A,-
97951,17,78919466.0,78919466.0,G,T
97952,17,15967388.0,15967388.0,G,A
97953,16,30129462.0,30129462.0,C,T
97954,16,339494.0,339494.0,C,T
97955,23,44918328.0,44918328.0,C,T


In [134]:
maf_cohort.to_pickle(data_path + 'merged_data/maf_cohort_annotated.pkl')